In [2]:
# Import of relevant libraries and loading the Our world in Data Database

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

file_path = '20800/owid-covid-data.csv'  # Replace with your file path
covid_data = pd.read_csv(file_path)


FileNotFoundError: [Errno 2] No such file or directory: '/shiv_folder/'

In [ ]:
# Filtering the data for the specified countries and time period (focus on initial stages)
countries = ['United States', 'Italy', 'France']
time_period = (pd.to_datetime("2020-03-01"), pd.to_datetime("2020-06-30"))

# Selecting relevant columns and rows
filtered_data = covid_data[covid_data['location'].isin(countries) & 
                           (covid_data['date'] >= time_period[0].strftime('%Y-%m-%d')) & 
                           (covid_data['date'] <= time_period[1].strftime('%Y-%m-%d'))].copy()

# Convert 'new_tests_smoothed_per_thousand' to per million
filtered_data['new_tests_smoothed_per_million'] = filtered_data['new_tests_smoothed_per_thousand'] * 1000

# Columns to use for the plot
required_metrics = ['new_tests_smoothed_per_million', 'new_cases_smoothed_per_million', 'new_deaths_smoothed_per_million']

# Melting the data for visualization
melted_data = filtered_data.melt(id_vars=['location', 'date'], 
                                 value_vars=required_metrics, 
                                 var_name='Metric', 
                                 value_name='Value')

In [ ]:
# Gra
# Replacing column names for the plot
metric_name_replacements = {
    'new_tests_smoothed_per_million': 'Tests per Million',
    'new_cases_smoothed_per_million': 'Cases per Million',
    'new_deaths_smoothed_per_million': 'Deaths per Million'
}
melted_data.replace({"Metric": metric_name_replacements}, inplace=True)

# Converting the date column to month names for the plot
melted_data['Month'] = pd.to_datetime(melted_data['date']).dt.strftime('%B')

# Creating the FacetGrid plot with Month as rows and countries as the x-axis
g = sns.FacetGrid(melted_data, col="Month", hue="Metric", sharey=False, height=6, aspect=1)
g.map(sns.barplot, "location", "Value", order=countries, edgecolor="black")

# Adding titles and labels
g.set_titles("{col_name}")
g.set_axis_labels("", "Value per Million")
g.add_legend(title="Metric")
plt.subplots_adjust(top=0.9)
g.fig.suptitle('COVID-19 Tests, Cases, and Deaths per million | Comparison by Month and Country (March-June 2020)')
plt.show()


In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Load the provided COVID-19 dataset
file_path = 'covid_19_clean_complete.csv' 
covid_data_map = pd.read_csv(file_path)

# Calculating death rate (deaths/confirmed) by country
covid_data_map['Death Rate'] = covid_data_map['Deaths'] / covid_data_map['Confirmed']
covid_data_map_grouped = covid_data_map.groupby('Country/Region')[['Confirmed', 'Deaths', 'Death Rate']].sum().reset_index()

# Load a world map for plotting
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Merging the COVID-19 data with the world map data
world_covid = world.merge(covid_data_map_grouped, how='left', left_on='name', right_on='Country/Region')

# Setting NaN values in 'Death Rate' to a value that indicates no data (for coloring purposes)
world_covid['Death Rate'] = world_covid['Death Rate'].fillna(-1)

# Plotting with adjusted color scheme
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
world_covid.plot(column='Death Rate', ax=ax, legend=True, cmap='OrRd', 
                 missing_kwds={'color': 'lightgrey', 'label': 'No Data'},
                 legend_kwds={'label': "COVID-19 Death Rate by Country",
                              'orientation': "horizontal"})
plt.title('Global COVID-19 Death Rate by Country')
plt.show()


In [ ]:
from scipy.stats import linregress
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'Tests' is represented by 'Confirmed' cases as a proxy
covid_hdi_merged['Tests'] = covid_hdi_merged['Confirmed']

# Extracting HDI and Confirmed cases (as a proxy for tests) for linear regression analysis
x = covid_hdi_merged['HDI']
y = covid_hdi_merged['Tests']

# Performing linear regression
regression_result = linregress(x, y)

# Calculating the R-squared value
r_squared_tests = regression_result.rvalue ** 2

# Creating a scatter plot with regression line
plt.figure(figsize=(10, 6))
sns.regplot(x=x, y=y, scatter_kws={'alpha':0.5}, line_kws={'color': 'red'})
plt.title('Correlation between COVID-19 Tests and Human Development Index (HDI) by Country')
plt.xlabel('Human Development Index (HDI)')
plt.ylabel('COVID-19 Tests (Approximated by Confirmed Cases)')
plt.show()

# Output the R-squared value
print("R-squared value:", r_squared_tests)


In [ ]:
from scipy.stats import linregress

# Extracting HDI and Death Rate data for linear regression analysis
x = covid_hdi_merged['HDI']
y = covid_hdi_merged['Death Rate']

# Performing linear regression
regression_result = linregress(x, y)

# Calculating the R-squared value
r_squared = regression_result.rvalue ** 2
r_squared


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import linregress
import geopandas as gpd

# Load and preprocess COVID-19 testing data
file_path_testing = 'owid-covid-data.csv'
covid_testing_data = pd.read_csv(file_path_testing)
covid_testing_grouped = covid_testing_data.groupby('location').agg({'total_tests': 'max', 'population': 'max'}).reset_index()

# Calculating tests per person
covid_testing_grouped['Tests Per Person'] = covid_testing_grouped['total_tests'] / covid_testing_grouped['population']

# Load and preprocess HDI data
file_path_hdi = 'human-development-index-(hdi)-by-country-2024.csv'
hdi_data = pd.read_csv(file_path_hdi)
hdi_data.rename(columns={'country': 'Country', 'Hdi2020': 'HDI'}, inplace=True)

# Merging HDI data with COVID-19 testing data
covid_testing_hdi_merged = covid_testing_grouped.merge(hdi_data[['Country', 'HDI']], left_on='location', right_on='Country', how='inner')

# Exclude countries with missing test data
covid_testing_hdi_filtered = covid_testing_hdi_merged.dropna(subset=['Tests Per Person'])

# Applying log scale to tests per person for comparison with HDI
covid_testing_hdi_filtered['Log Tests Per Person'] = np.log(covid_testing_hdi_filtered['Tests Per Person'] + 1) # Adding 1 to avoid log(0)

# Scatter plot and R-squared for log-scaled tests per person and HDI
x_log_tests = covid_testing_hdi_filtered['HDI']
y_log_tests = covid_testing_hdi_filtered['Log Tests Per Person']

# Linear regression on log-transformed data
regression_result_log_tests = linregress(x_log_tests, y_log_tests)

# R-squared value for the log-transformed data
r_squared_log_tests = regression_result_log_tests.rvalue ** 2

# Scatter plot
plt.figure(figsize=(10, 6))
sns.regplot(x=x_log_tests, y=y_log_tests, scatter_kws={'alpha':0.5}, line_kws={'color': 'red'})
plt.title('Correlation between Log of COVID-19 Tests Per Person and HDI by Country')
plt.xlabel('Human Development Index (HDI)')
plt.ylabel('Log of COVID-19 Tests Per Person')
plt.show()

print("R-squared value:", r_squared_log_tests)

# Preparing data for geographical mapping with log tests per person and HDI ratio
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world_covid_tests_per_person_hdi = world.merge(covid_testing_hdi_filtered, how='left', left_on='name', right_on='Country')

# Ratio and log transformation for the map
world_covid_tests_per_person_hdi['Tests Per Person/HDI Ratio'] = world_covid_tests_per_person_hdi['Tests Per Person'] / world_covid_tests_per_person_hdi['HDI']
world_covid_tests_per_person_hdi['Log Tests Per Person/HDI Ratio'] = np.log(world_covid_tests_per_person_hdi['Tests Per Person/HDI Ratio'] + 0.0001)

# Plotting the map
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
world_covid_tests_per_person_hdi.plot(column='Log Tests Per Person/HDI Ratio', ax=ax, legend=True, cmap='YlGnBu', 
                                      missing_kwds={'color': 'lightgrey', 'label': 'No Data'},
                                      legend_kwds={'label': "Log of Tests Per Person/HDI Ratio",
                                                   'orientation': "horizontal"})
plt.title('Global COVID-19 Tests Per Person to HDI Ratio by Country (Log Scale)')
plt.show()


Higher color intensity indicates a higher log-scaled ratio of tests to HDI, suggesting more testing in relation to the country's development level.
This visualization can help understand how testing strategies and capabilities vary with the level of human development across countries.
Countries with higher HDI might have more resources and infrastructure to conduct extensive testing, which is crucial for managing the pandemic.
Countries in light grey indicate no data available for either testing or HDI.